In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
pd.set_option('display.max_columns', None)

Посмотрим как **выглядит** наш датафрейм. Для этого считаем его с **csv-файла** и выведем первые **10 строк**

После выведем количество **строк и столбцов**

In [249]:
tbank_df = pd.read_csv('sale_tasks_dataset.csv')
tbank_df.head()

,customer_id,age,gender_cd,region_size,foreign_flg,entrepreneur_flg,auto_flg,traveler_flg,partnership_flg,child_amt,realty_type,segment_cd,bundle_nm,confirmed_income,predicted_income,product_cd,functional_cd,finish_dttm,state_talk_time_sec,wo_hit_status_result_desc,wo_hit_status_reason_desc,employee_id,emp_age,emp_gender_cd,emp_children_cnt,emp_work_months_cnt,emp_citizenship_country_cd,emp_marital_status_cd
0,422206773,58.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Debit Cards,Meeting,2022-04-12 07:00:00,67.0,"Дозвон, Отказ",Перезвонит самостоятельно позднее,142052.0,30,W,NaN,1,NaN,NaN
1,348648289,44.0,M,rural,0.0,0.0,0.0,0.0,1.0,1.0,Квартира,r_01 (6),Pro,NaN,NaN,Cash Loan,Upsell,2023-10-25 20:00:00,151.0,"Дозвон, Отказ",Другое,7004.0,36,W,NaN,2,RUS,NaN
2,345496995,23.0,F,town,0.0,0.0,1.0,0.0,1.0,0.0,Нет своего жилья,t_05 (19),XXX,NaN,NaN,MVNO,Afterfilling,2023-06-16 07:00:00,68.0,"Дозвон, Успешно",Назначена встреча,80760.0,52,W,NaN,6,NaN,NaN
3,182783192,34.0,M,town,0.0,0.0,0.0,0.0,0.0,0.0,Нет своего жилья,t_07 (29),NaN,NaN,NaN,Investment,Utilization,2022-09-23 12:00:00,738.0,"Дозвон, Успешно",Обещал утилизироваться самостоятельно,158672.0,22,NaN,NaN,8,NaN,NaN
4,138498254,30.0,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash Loan,Incoming,2023-02-27 16:00:00,39.0,"Дозвон, Отказ",Переведен в банк,14462.0,51,W,NaN,7,NaN,NaN


In [251]:
print(f'Количество строк и столбцов соответственно - {tbank_df.shape}')

Количество строк и столбцов соответственно - (18691, 28)


Значит, всего было совершено **18691 звонков**. Далее выведем **всю** информацию о датафрейме с помощью метода `info()`. Он покажет, где и сколько значение **non-null**, выведет тип **каждой колонки**

In [254]:
tbank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18691 entries, 0 to 18690
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_id                 18691 non-null  int64  
 1   age                         18509 non-null  object 
 2   gender_cd                   18433 non-null  object 
 3   region_size                 11402 non-null  object 
 4   foreign_flg                 11402 non-null  float64
 5   entrepreneur_flg            11402 non-null  float64
 6   auto_flg                    11402 non-null  float64
 7   traveler_flg                11402 non-null  float64
 8   partnership_flg             11402 non-null  float64
 9   child_amt                   11402 non-null  float64
 10  realty_type                 11402 non-null  object 
 11  segment_cd                  11402 non-null  object 
 12  bundle_nm                   4703 non-null   object 
 13  confirmed_income            140

Далее нам интересно сколько звонков **положительных**, а сколько **отрицательных**. Сначала узнаем какое количество **уникальных значений** есть в колонке результата

In [257]:
print(f'Количество уникальных значений - {tbank_df['wo_hit_status_result_desc'].nunique()}')

Количество уникальных значений - 2


Так как их всего 2, мы можем выполнить **унитарное преобразование** (one-hot encoding) и посчитать **количество отказов** и **успещных** звонков

In [260]:
one_hot_df = pd.get_dummies(tbank_df['wo_hit_status_result_desc'], prefix='wo_hit_status_result_desc', drop_first=True, dtype='int')
count_calls = {0 : one_hot_df.value_counts()[0], 1 : one_hot_df.value_counts()[1]}
print(f'Количество успешных звонков - {count_calls[1]}, количество отказов - {count_calls[0]}')
print(f'Период, указанный в датафрейме: {pd.to_datetime(min(tbank_df.finish_dttm)).strftime('%m.%y')} - {pd.to_datetime(max(tbank_df.finish_dttm)).strftime('%m.%y')}')

Количество успешных звонков - 2417, количество отказов - 16274
Период, указанный в датафрейме: 01.22 - 11.24


Так же выведем информацию о том, за какой **период** у нас есть данные **(январь 2022 - ноябрь 2024)**

Выполним унитарное преобразование и для других колонок (в дальнейшем буду называть их **признаки**, числовые колонки - это **вещественные** признаки, колонки типа object - **категориальные**)

Преобразуем так признаки: **пол** (gender_cd, female - 0, male - 1), **наличие жилья** (realty_type, есть - 1 или нет его - 0), **населенный пункт** (urban, town, msk - город, rural	- сельская местность). Так же переопределим признак **подписки** (Pro и Premium - без изменений, XXX - нет подписки)

Добавим в датафрейм признак на колонку результата звонка

In [263]:
tbank_df['realty_type'] = tbank_df['realty_type'].map({'Жилой дом' : 1,
                                                       'Квартира' : 1,
                                                       'Комната' : 0,
                                                       'Нет своего жилья' : 0
                                                      })
tbank_df['region_size'] = tbank_df['region_size'].map({
    'urban' : 'город',
    'town' : 'город',
    'msk' : 'город',
    'rural' : 'село'
})
tbank_df['gender_cd'] = tbank_df['gender_cd'].map({
    'F' : 0,
    'M' : 1
})
tbank_df['bundle_nm'] = tbank_df['bundle_nm'].map({
    'Premium' : 'Premium',
    'Pro' : 'Pro',
    'XXX' : 'Без подписки'
})
tbank_df['result_calling'] = tbank_df['wo_hit_status_result_desc'].map({
    'Дозвон, Отказ' : 0,
    'Дозвон, Успешно' : 1
})

In [265]:
set(tbank_df['product_cd'])

{'Car Loan',
 'Casco',
 'Cash Loan',
 'Credit Cards',
 'Debit Cards',
 'Deposit',
 'Investment',
 'MVNO',
 'Mortgage Insurance',
 'Osago',
 'Secured Loan',
 'T-Auto',
 'Travel Avia'}

Теперь добавим стобец, по поводу какого **продукта** звонили. Всего продуктов у нас - **13** (все они представлены выше), разобьем их на **когорты**: `Кредитные карты`, `Кредиты`, `Дебетовые карты`, `Инвестиции`, `Услуги и сервисы`, `Страхование` и `Другое`

Значений **NaN** в product_cd у нас нет, поэтому все будет работать **корректно**

In [268]:
credit_cards = ['Credit Cards']
credit = ['Cash Loan', 'Car Loan', 'Secured Loan']
debet_cards = ['Debit Cards']
invest = ['Investment']
insurance = ['Casco', 'Osago', 'Mortgage Insurance']
services = ['T-Auto', 'Travel Avia']
tbank_df['product'] = 'Другое'
tbank_df.loc[tbank_df['product_cd'].isin(credit_cards), 'product'] = 'Кредитная карта'
tbank_df.loc[tbank_df['product_cd'].isin(credit), 'product'] = 'Кредит'
tbank_df.loc[tbank_df['product_cd'].isin(debet_cards), 'product'] = 'Дебетовая карта'
tbank_df.loc[tbank_df['product_cd'].isin(insurance), 'product'] = 'Страхование'
tbank_df.loc[tbank_df['product_cd'].isin(invest), 'product'] = 'Инвестиции'
tbank_df.loc[tbank_df['product_cd'].isin(services), 'product'] = 'Услуги и сервисы'

Распределим клиентов по **категориям**, кто к какой возростной группе принадлежит. Будем делить следующим образом: `14-25`, `26-35`, `36-50`, `51-65`, `65+` (Поясню: можно сделать через `pd.cut`, но на **больших данных** может медленно работать)

Так же можно добавить столбец **разницы** в возрасте между **пользователем и оператором** колл-центра

In [271]:
tbank_df['age'] = tbank_df['age'].replace(',', '.', regex=True)
tbank_df['age'] = tbank_df['age'].astype('float')
conditions = [
    (tbank_df['age'] <= 25),
    (tbank_df['age'].between(26, 35)),
    (tbank_df['age'].between(36, 50)),
    (tbank_df['age'].between(51, 65)),
    (tbank_df['age'] > 65)
]
choices = ['14-25', '26-35', '36-50', '51-65', '65+']
tbank_df['age_group'] = np.select(conditions, choices, default='unknown')

tbank_df['age_difference'] = np.abs(tbank_df['emp_age'] - tbank_df['age'])

In [273]:
set(tbank_df['functional_cd'])

{'Activation',
 'Afterfilling',
 'Agreement',
 'Autoleads',
 'BAF',
 'Cold',
 'Deposit',
 'Downsell',
 'Incoming',
 'Meeting',
 'Preapprove',
 'Prolongation',
 'Refinancing',
 'Reutilization',
 'Upsell',
 'Utilization'}

Теперь сделаем отдельно столбец по **причине звонка**. Один - `Предложения и продажи`, другой - `Другое`

In [276]:
loan_pr = ['Upsell', 'Cold', 'Downsell', 'Utilization', 'Reutilization', 'Prolongation', 'Refinancing', 'Preapprove', 'Autoleads', 'BAF']

tbank_df['functional'] = 'Другое'
tbank_df.loc[tbank_df['functional_cd'].isin(loan_pr), 'functional'] = 'Предложения и продажи'

Так же заполним **пропуски** в признаке `bundle_nm` значением `Без подписки`

Посмотрим сколько клиентов с каждой подпиской. Это пригодится для дальнейшего тестирования гипотез

In [279]:
tbank_df["bundle_nm"] = tbank_df["bundle_nm"].fillna('Без подписки')
b = np.array(tbank_df['bundle_nm'])
count_bundle = Counter(b)
print(f'Без подписки - {count_bundle['Без подписки']}, Pro - {count_bundle['Pro']}, Premium - {count_bundle['Premium']}')

Без подписки - 13989, Pro - 4406, Premium - 296


В дальнейшем будем анализировать клиентов **без подписки и с подпиской** Pro, Premium - убираем, так как очень мало данных

Проверим на **дубликаты** `customer_id` и удалим, если есть

In [282]:
tbank_df = tbank_df[tbank_df['bundle_nm'].isin(['Без подписки', 'Pro'])]
tbank_df['with_pro'] = tbank_df['bundle_nm'].map({
    'Без подписки' : 0,
    'Pro' : 1
})

In [284]:
tbank_df.customer_id.nunique()

17012

Видим, что есть дубликаты, поэтому **удалим** их

In [287]:
tbank_df.drop_duplicates(subset=['customer_id'], inplace=True, keep='first')

Теперь уберем выбросы в длительности разговора

In [290]:
q25, q75 = tbank_df['state_talk_time_sec'].quantile([0.25, 0.75])
IQR = q75 - q25
high = q75 + 1.5 * IQR # coefficient 1.5 - standart
tbank_df = tbank_df[tbank_df['state_talk_time_sec'].between(0, high, inclusive='neither')]
tbank_df.shape

(15418, 34)

In [306]:
print(f'Количество иностранцев - {Counter(np.array(tbank_df.foreign_flg))[1]}')

Количество иностранцев - 83


Так же уберем из нашего датафрейма данные об **иностранцах** - их всего 83 человека. В признаке **иностранец или нет** много пропусков, не будем их ничем заполнять, а просто удалим их, так как, если **проанализировать данные**, то если нет информации **иностранец человек или нет**, то многих других признаков у этого объекта тоже **не будет**

In [330]:
tbank_df_new = tbank_df[tbank_df['foreign_flg'] == 0]
tbank_df_new.head(15)

,customer_id,age,gender_cd,region_size,foreign_flg,entrepreneur_flg,auto_flg,traveler_flg,partnership_flg,child_amt,realty_type,segment_cd,bundle_nm,confirmed_income,predicted_income,product_cd,functional_cd,finish_dttm,state_talk_time_sec,wo_hit_status_result_desc,wo_hit_status_reason_desc,employee_id,emp_age,emp_gender_cd,emp_children_cnt,emp_work_months_cnt,emp_citizenship_country_cd,emp_marital_status_cd,result_calling,product,age_group,age_difference,functional,with_pro
1,348648289,44.0,1.0,село,0.0,0.0,0.0,0.0,1.0,1.0,1.0,r_01 (6),Pro,NaN,NaN,Cash Loan,Upsell,2023-10-25 20:00:00,151.0,"Дозвон, Отказ",Другое,7004.0,36,W,NaN,2,RUS,NaN,0,Кредит,36-50,8.0,Предложения и продажи,1
2,345496995,23.0,0.0,город,0.0,0.0,1.0,0.0,1.0,0.0,0.0,t_05 (19),Без подписки,NaN,NaN,MVNO,Afterfilling,2023-06-16 07:00:00,68.0,"Дозвон, Успешно",Назначена встреча,80760.0,52,W,NaN,6,NaN,NaN,1,Другое,14-25,29.0,Другое,0
5,64946855,31.0,1.0,город,0.0,0.0,0.0,0.0,1.0,0.0,1.0,t_07 (29),Без подписки,NaN,15362.87,Cash Loan,Preapprove,2023-11-24 07:00:00,36.0,"Дозвон, Отказ",Отказ прослушать предложение,184241.0,43,W,NaN,6,RUS,UNM,0,Кредит,26-35,12.0,Предложения и продажи,0
6,516320897,34.0,0.0,город,0.0,0.0,0.0,0.0,1.0,0.0,0.0,t_10 (40),Без подписки,NaN,13233.30,Mortgage Insurance,Cold,2024-08-17 12:00:00,38.0,"Дозвон, Отказ",Страхуется в своей СК,106508.0,74,M,NaN,1,RUS,NaN,0,Страхование,26-35,40.0,Предложения и продажи,0
8,148619460,67.0,0.0,город,0.0,0.0,0.0,0.0,1.0,0.0,1.0,t_08 (30),Без подписки,NaN,NaN,Credit Cards,Preapprove,2022-07-27 14:00:00,24.0,"Дозвон, Отказ",Отказ прослушать предложение,104002.0,46,W,NaN,10,NaN,NaN,0,Кредитная карта,65+,21.0,Предложения и продажи,0
9,67061355,41.0,0.0,город,0.0,1.0,1.0,0.0,1.0,1.0,1.0,u_04 (14),Без подписки,NaN,NaN,Cash Loan,Upsell,2023-08-28 10:00:00,22.0,"Дозвон, Отказ",Другое,209308.0,20,W,NaN,7,NaN,NaN,0,Кредит,36-50,21.0,Предложения и продажи,0
10,299782696,37.0,1.0,город,0.0,0.0,1.0,0.0,1.0,1.0,1.0,u_01 (3),Без подписки,NaN,NaN,MVNO,Afterfilling,2022-11-08 12:00:00,20.0,"Дозвон, Отказ",Другое,14829.0,58,W,NaN,11,NaN,NaN,0,Другое,36-50,21.0,Другое,0
12,134682688,36.0,0.0,город,0.0,0.0,0.0,0.0,1.0,0.0,1.0,m_07 (18),Pro,NaN,NaN,Investment,Reutilization,2022-11-22 15:00:00,28.0,"Дозвон, Отказ",Отказ от диалога,169086.0,32,M,NaN,4,NaN,NaN,0,Инвестиции,36-50,4.0,Предложения и продажи,1
15,383906888,18.0,1.0,город,0.0,0.0,0.0,0.0,0.0,0.0,1.0,t_09 (36),Без подписки,NaN,NaN,Credit Cards,Utilization,2023-05-29 10:00:00,31.0,"Дозвон, Отказ",Отказ от использования КК,40894.0,41,W,NaN,6,NaN,NaN,0,Кредитная карта,14-25,23.0,Предложения и продажи,0
17,158045014,38.0,1.0,город,0.0,0.0,1.0,0.0,1.0,0.0,1.0,u_04 (14),Без подписки,NaN,NaN,Investment,Utilization,2023-08-08 09:00:00,45.0,"Дозвон, Отказ",Не звонить,180298.0,23,M,NaN,6,RUS,NaN,0,Инвестиции,36-50,15.0,Предложения и продажи,0


Сохраним **новый датафрейм**, в будущем, возможно, добавятся **новые колонки**, но это не точно)

In [332]:
tbank_df_new.to_csv('new_df.csv', index=False)